## Importing packages and data

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import xgboost as xgb ##download xgboost using "pip3 install xgboost"

# data = pd.read_csv('form_responses.csv')

# pip install sdv to download sdv package

## Loading data

In [5]:
data = pd.read_csv('cleaned_routes.csv')
data

,ISB_Service,bus_stop_board,bus_stop_alight,day_of_the_week,time_start,travel_duration,frequency,punctuality,cleanliness,safety,coverage,crowdedness
0,A2,PGP Foyer,Ventus,Monday,09:30:00,15 - 20 minutes,Neutral,Neutral,Very Satisfied,Satisfied,Satisfied,3.0
1,D2,KR MRT,UTown,Monday,09:30:00,10 - 15 minutes,Satisfied,Satisfied,Satisfied,Satisfied,Satisfied,4.0
2,A2,KR Bus Terminal,S17,Thursday,17:50:00,< 5 minutes,Very Satisfied,Satisfied,Satisfied,Satisfied,Very Satisfied,4.0
3,A1,PGP Terminal,University Hall,Friday,11:30:00,10 - 15 minutes,Very Satisfied,Very Satisfied,Very Satisfied,Neutral,Very Satisfied,4.0
4,A1,KR MRT,LT27,Tuesday,07:50:00,< 5 minutes,Neutral,Satisfied,Satisfied,Satisfied,Satisfied,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
285,A1,BIZ2,LT27,Monday,11:40:00,10 - 15 minutes,Neutral,Satisfied,Very Satisfied,Very Satisfied,Very Satisfied,5.0
286,D2,KR MRT,TCOMS,Wednesday,08:45:45,10 - 15 minutes,Dissatisfied,Satisfied,Very Satisfied,Neutral,Dissatisfied,4.0
287,D2,S17,COM3,Thursday,07:45:45,10 - 15 minutes,Neutral,Very Satisfied,Satisfied,Very Satisfied,Satisfied,4.0
288,A2,IT,Opp HSSML,Monday,10:45:45,< 5 minutes,Dissatisfied,Satisfied,Satisfied,Neutral,Satisfied,4.0


## Creating metadata

### for routes only

In [10]:
from sdv.metadata import Metadata

metadata = Metadata.detect_from_dataframe(
    data = data,
    table_name='transport')

metadata.update_column(
    column_name='time_start',
    sdtype='datetime',
    datetime_format= '%H:%M:%S')

metadata.validate()

metadata


{
    "tables": {
        "transport": {
            "columns": {
                "ISB_Service": {
                    "sdtype": "categorical"
                },
                "bus_stop_board": {
                    "sdtype": "categorical"
                },
                "bus_stop_alight": {
                    "sdtype": "categorical"
                },
                "day_of_the_week": {
                    "sdtype": "categorical"
                },
                "time_start": {
                    "sdtype": "datetime",
                    "datetime_format": "%H:%M:%S"
                },
                "travel_duration": {
                    "sdtype": "categorical"
                },
                "frequency": {
                    "sdtype": "categorical"
                },
                "punctuality": {
                    "sdtype": "categorical"
                },
                "cleanliness": {
                    "sdtype": "categorical"
                },
          

## Utilising GaussianCopulaSynthesizer

In [40]:
from sdv.single_table import GaussianCopulaSynthesizer

synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.load_custom_constraint_classes(filepath='custom_constraints.py', class_names=['BusStopsCheck', 'TimeCheck'])

route_constraint = {
    'constraint_class': 'BusStopsCheck',
    'constraint_parameters': {
        'column_names':['ISB_Service', 'bus_stop_board', 'bus_stop_alight']
    }
}
time_constraint = {
    'constraint_class': 'TimeCheck',
    'constraint_parameters': {
        'column_names':['time_start']
    }
}

synthesizer.add_constraints(constraints=[route_constraint, time_constraint])

synthesizer.fit(data)

synthetic_data = synthesizer.sample(num_rows=10000)

synthetic_data

Sampling rows: 100%|██████████| 10000/10000 [00:03<00:00, 2852.65it/s]


,ISB_Service,bus_stop_board,bus_stop_alight,day_of_the_week,time_start,travel_duration,frequency,punctuality,cleanliness,safety,coverage,crowdedness
0,D2,KR MRT,UTown,Monday,08:17:43,5 - 10 minutes,Satisfied,Satisfied,Satisfied,Neutral,Satisfied,4.0
1,D2,KR MRT,LT27,Thursday,17:15:53,< 5 minutes,Neutral,Satisfied,Satisfied,Satisfied,Satisfied,5.0
2,D2,Opp KR MRT,PGP Foyer,Thursday,08:09:21,10 - 15 minutes,Neutral,Satisfied,Satisfied,Neutral,Very Satisfied,5.0
3,D1,YIH,COM3,Thursday,08:09:02,5 - 10 minutes,Dissatisfied,Satisfied,Satisfied,Very dissatisfied,Very dissatisfied,5.0
4,A1,KR MRT,CLB,Wednesday,10:28:37,5 - 10 minutes,Dissatisfied,Satisfied,Satisfied,Dissatisfied,Satisfied,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,D1,Museum,CLB,Monday,15:12:48,5 - 10 minutes,Dissatisfied,Dissatisfied,Satisfied,Very dissatisfied,Neutral,2.0
9996,A1,BIZ2,CLB,Tuesday,20:45:45,20 - 30 minutes,Neutral,Very dissatisfied,Very Satisfied,Neutral,Satisfied,2.0
9997,D2,KR MRT,UTown,Wednesday,20:45:45,5 - 10 minutes,Neutral,Very Satisfied,Very Satisfied,Neutral,Satisfied,4.0
9998,A1,KR MRT,KR Bus Terminal,Thursday,09:28:01,5 - 10 minutes,Dissatisfied,Satisfied,Satisfied,Dissatisfied,Satisfied,5.0


In [41]:
synthesizer.get_learned_distributions()

{'ISB_Service': {'distribution': 'beta',
  'learned_parameters': {'loc': 0.0017874543315022971,
   'scale': 0.9974191713480585,
   'a': 0.9017274626243743,
   'b': 0.9755642462209718}},
 'bus_stop_board': {'distribution': 'beta',
  'learned_parameters': {'loc': 0.005620606284709666,
   'scale': 0.9916506796214529,
   'a': 1.0216243143658579,
   'b': 0.9819670145534691}},
 'bus_stop_alight': {'distribution': 'beta',
  'learned_parameters': {'loc': 0.001462034630041732,
   'scale': 0.9980722941023545,
   'a': 0.9536277283228882,
   'b': 0.961806198352781}},
 'day_of_the_week': {'distribution': 'beta',
  'learned_parameters': {'loc': 0.004986229126805837,
   'scale': 0.9922618618090253,
   'a': 0.9584505612531116,
   'b': 0.9812253433299487}},
 'time_start': {'distribution': 'beta',
  'learned_parameters': {'loc': -2.2089609000000003e+18,
   'scale': 48015535358267.91,
   'a': 0.8244999308269605,
   'b': 1.282696263137874}},
 'travel_duration': {'distribution': 'beta',
  'learned_paramete

## Utilising CTGAN

In [42]:
from sdv.single_table import CTGANSynthesizer

ctgan = CTGANSynthesizer(metadata)

ctgan.load_custom_constraint_classes(filepath='custom_constraints.py', class_names=['BusStopsCheck', 'TimeCheck'])

route_constraint = {
    'constraint_class': 'BusStopsCheck',
    'constraint_parameters': {
        'column_names':['ISB_Service', 'bus_stop_board', 'bus_stop_alight']
    }
}
time_constraint = {
    'constraint_class': 'TimeCheck',
    'constraint_parameters': {
        'column_names':['time_start']
    }
}

ctgan.add_constraints(constraints=[route_constraint, time_constraint])

ctgan.fit(data)

ctgan_synthetic_data = ctgan.sample(num_rows=10000)

ctgan_synthetic_data

Sampling rows: 100%|██████████| 10000/10000 [00:03<00:00, 3129.80it/s]


,ISB_Service,bus_stop_board,bus_stop_alight,day_of_the_week,time_start,travel_duration,frequency,punctuality,cleanliness,safety,coverage,crowdedness
0,D2,University Hall,Opp KR MRT,Monday,12:35:56,15 - 20 minutes,Satisfied,Satisfied,Satisfied,Very Satisfied,Dissatisfied,5.0
1,D2,UTown,TCOMS,Friday,08:36:47,10 - 15 minutes,Satisfied,Satisfied,Very Satisfied,Satisfied,Very Satisfied,1.0
2,A1,KR MRT,CLB,Tuesday,08:03:13,20 - 30 minutes,Satisfied,Very Satisfied,Neutral,Very Satisfied,Satisfied,5.0
3,A2,IT,Opp KR MRT,Thursday,07:45:00,10 - 15 minutes,Satisfied,Very dissatisfied,Very Satisfied,Satisfied,Satisfied,1.0
4,A2,S17,KR Bus Terminal,Monday,09:25:11,10 - 15 minutes,Dissatisfied,Satisfied,Satisfied,Satisfied,Satisfied,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,D2,LT27,Opp KR MRT,Monday,09:30:00,5 - 10 minutes,Satisfied,Satisfied,Very Satisfied,Satisfied,Satisfied,5.0
9996,D2,KR MRT,COM3,Wednesday,08:11:30,10 - 15 minutes,Satisfied,Satisfied,Very Satisfied,Satisfied,Satisfied,2.0
9997,D2,KR MRT,UTown,Sunday,08:00:47,< 5 minutes,Satisfied,Satisfied,Very Satisfied,Neutral,Satisfied,1.0
9998,A1,KR MRT,LT27,Monday,07:45:00,20 - 30 minutes,Dissatisfied,Neutral,Very Satisfied,Dissatisfied,Satisfied,5.0


In [43]:
fig = ctgan.get_loss_values_plot()
fig.show()

## Validate synthetic data

In [25]:
A1_bus = ['KR Bus Terminal', 'LT13', 'AS5', 'BIZ2', 'Opp TCOMS', 'PGP Terminal', 'KR MRT', 'LT27', 'University Hall', 'Opp UHC', 'YIH', 'CLB', 'KR Bus Terminal']
A2_bus = ['KR Bus Terminal', 'IT', 'Opp YIH', 'Museum', 'UHC', 'Opp University Hall', 'S17', 'Opp KR MRT', 'PGP Foyer', 'TCOMS', 'Opp HSSML', 'Opp NUSS', 'Ventus', 'KR Bus Terminal']
D1_bus = ['COM3', 'Opp HSSML', 'Opp NUSS', 'Ventus', 'IT', 'Opp YIH', 'Museum', 'UTown', 'YIH', 'CLB', 'LT13', 'AS5', 'BIZ2', 'COM3']
D2_bus = ['COM3', 'Opp TCOMS', 'PGP Terminal', 'KR MRT', 'LT27', 'University Hall', 'Opp UHC', 'Museum', 'UTown', 'UHC', 'Opp University Hall', 'S17', 'Opp KR MRT', 'PGP Foyer', 'TCOMS', 'COM3']
BTC_bus = ['Oei Tiong Ham Building (BTC)', 'Botanic Gardens MRT (BTC)', 'KR MRT', 'LT27', 'University Hall', 'Opp UHC', 'UTown', 'Raffles Hall', 'Kent Vale', 'Museum', 'YIH', 'CLB', 'LT13', 'AS5', 'BIZ2', 'PGP Terminal', 'College Green (BTC)', 'Oei Tiong Ham Building (BTC)']
E_bus = ['UTown', 'Raffles Hall', 'Kent Vale', 'EA', 'SDE3', 'IT', 'Opp YIH', 'UTown']
K_bus = ['PGP Terminal', 'KR MRT', 'LT27', 'University Hall', 'Opp UHC', 'YIH', 'CLB', 'Opp SDE3', 'The Japanese Primary School', 'Kent Vale', 'Museum', 'UHC', 'Opp University Hall', 'S17', 'Opp KR MRT', 'PGP Foyer']
L_bus = ['Oei Tiong Ham Building (BTC)', 'Botanic Gardents MRT (BTC)', 'College Green (BTC)', 'Oei Tiong Ham Building (BTC)']

bus_routes = {'A1':A1_bus, 'A2':A2_bus, 'D1':D1_bus, 'D2':D2_bus, 'BTC (Bukit Timah Campus)':BTC_bus, 'E':E_bus, 'K':K_bus, 'L':L_bus}

def validate_bus_stops(data, bus_routes):
    service_col = 'ISB_Service'
    board_col = 'bus_stop_board'
    alight_col = 'bus_stop_alight'

    def check_stops(row):
        bus = row[service_col]
        start = row[board_col]
        end = row[alight_col]

        # Get the route for the bus
        route = bus_routes.get(bus, [])

        if start in route and end in route:
            start_index = route.index(start)
            end_index = route.index(end) if end != route[0] else len(route)

            if start_index < end_index:
                return bus, start, end  # No change if valid

        # If only start is valid, randomly select an end stop from the route after start
        if start in route:
            start_index = route.index(start)
            if start_index < len(route) - 1:  # Ensure there are stops after start
                possible_ends = route[start_index + 1:]
                new_end = np.random.choice(possible_ends)
                return bus, start, new_end

        # If only end is valid, select a new start stop before end
        if end in route:
            end_index = route.index(end)
            if end_index == 0:
                end_index = len(route) - 1  # If end is the first stop, assume it is the last stop (loop)
            if end_index > 0:
                possible_starts = route[:end_index]
                new_start = np.random.choice(possible_starts)
                return bus, new_start, end

        # Check for valid buses that have both start and end in the route
        valid_buses = [
            key for key, route in bus_routes.items()
            if start in route and end in route and
            (route.index(start) < route.index(end) or
                (route.index(end) == 0 and route.index(start) < len(route) - 1))
        ]
        if valid_buses:
            new_bus = np.random.choice(valid_buses)  # Randomly choose one valid bus
            return new_bus, start, end

        # else, select two random stops in the correct order from the current bus route
        random_start_index = np.random.randint(0, len(route) - 1)
        random_end_index = np.random.randint(random_start_index + 1, len(route))
        new_start = route[random_start_index]
        new_end = route[random_end_index]
        return bus, new_start, new_end

    data[[service_col, board_col, alight_col]] = data.apply(lambda row: check_stops(row), axis=1, result_type="expand")

    return data

def check_validate_bus_stops(data, bus_routes):
    service_col = 'ISB_Service'
    board_col = 'bus_stop_board'
    alight_col = 'bus_stop_alight'

    def check_stops(row):
        bus = row[service_col]
        start = row[board_col]
        end = row[alight_col]

        # Check if bus route exists in the dictionary
        if bus in bus_routes:
            route = bus_routes[bus]

            # Check both stops are in the route and start is before end
            if start in route and end in route:
                start_index = route.index(start)
                # Handle looping
                end_index = route.index(end) if end != route[0] else len(route)

                if start_index < end_index:
                    return row[board_col], row[alight_col]  # No change if valid
        return "error", "error"
    data[[board_col, alight_col]] = data.apply(lambda row: check_stops(row), axis=1, result_type="expand")

    return data

synthetic_data = validate_bus_stops(synthetic_data, bus_routes)

check_data = check_validate_bus_stops(synthetic_data, bus_routes)
error_rows = check_data[check_data.isin(['error']).any(axis=1)]
error_rows
len(error_rows)

0

In [26]:
ctgan_synthetic_data = validate_bus_stops(ctgan_synthetic_data, bus_routes)

check_data = check_validate_bus_stops(ctgan_synthetic_data, bus_routes)
error_rows = check_data[check_data.isin(['error']).any(axis=1)]
error_rows
len(error_rows)

0

### Write synthetic data to CSV

In [ ]:
# synthetic_data.to_csv('synthetic_data.csv', index=False)

### For everything

In [24]:
from sdv.metadata import Metadata
from sdv.single_table import GaussianCopulaSynthesizer

allmetadata = Metadata.detect_from_dataframe(
    data = data,
    table_name='alltransport')


allmetadata.update_column(
    column_name='time_start_trip_1',
    sdtype='datetime',
    datetime_format= '%I:%M:%S %p' )


allmetadata.update_column(
    column_name='time_start_trip_2',
    sdtype='datetime',
    datetime_format= '%I:%M:%S %p' )


allmetadata.update_column(
    column_name='time_start_trip_2',
    sdtype='datetime',
    datetime_format= '%I:%M:%S %p' )



allmetadata.validate()


allmetadata

synthesizer1 = GaussianCopulaSynthesizer(allmetadata)
synthesizer1.fit(data)

synthetic_data = synthesizer1.sample(num_rows=50)

synthetic_data

c:\Users\enqil\AppData\Local\Programs\Python\Python310\lib\site-packages\sdv\single_table\base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


,timestamp,role,frequency_of_travel,primary_purpose,travel_days,travel_hours,ISB_Service_trip_1,bus_stop_board_trip_1,bus_stop_alight_trip_1,day_of_the_week_trip_1,time_start_trip_1,travel_duration_trip_1,frequency_trip_1,punctuality_trip_1,cleanliness_trip_1,safety_trip_1,coverage_trip_1,crowdedness_trip_1,ISB_Service_trip_2,bus_stop_board_trip_2,bus_stop_alight_trip_2,day_of_the_week_trip_2,time_start_trip_2,travel_duration_trip_2,frequency_trip_2,punctuality_trip_2,cleanliness_trip_2,safety_trip_2,coverage_trip_2,crowdedness_trip_2,ISB_Service_trip_3,bus_stop_board_trip_3,bus_stop_alight_trip_3,day_of_the_week_trip_3,time_start_trip_3,travel_duration_trip_3,frequency_trip_3,punctuality_trip_3,cleanliness_trip_3,safety_trip_3,coverage_trip_3,crowdedness_trip_3,usage_influence_convenience,usage_influence_cost,usage_influence_lack_of_options,usage_influence_availability_of_parking,usage_influence_environmental,prioritize_frequency,prioritize_punctuality,prioritize_cleanliness,prioritize_safety,prioritize_bus_route_coverage,prioritize_crowdedness,top_3_frustrations,not_able_to_get_on,additional_features_frequency,additional_features_seats,additional_features_cleanliness,additional_features_comfortable,additional_features_route_coverage,additional_features_updates,issues_with_quality_of_info,special_events,seasonal_changes,seasonal_changes_specific,further_comments
0,10/04/2024 04:49:04,Undergraduate student,3 - 4 days a week,Commute to classes,sdv-pii-i6eeb,sdv-pii-mhmvk,A1,KR MRT,Ventus,NaN,09:11:45 AM,10 - 15 minutes,Satisfied,Satisfied,Neutral,Dissatisfied,Satisfied,2,A2,sdv-pii-5n2x5,Opp KR MRT,NaN,NaN,15 - 20 minutes,Satisfied,Satisfied,NaN,NaN,Very Satisfied,1.0,D1,Botanic Gardens MRT (BTC),NaN,NaN,NaN,10 - 15 minutes,Neutral,Neutral,Neutral,NaN,NaN,NaN,2nd,3rd,1st,5th,4th,3rd,4th,4th,6th,5th,2nd,sdv-pii-yp0mw,Rarely,1st,3rd,4th,5th,5th,2nd,No,4,"Yes, service improves/worsens (please specify ...",NaN,NaN
1,10/02/2024 00:41:55,Staff,3 - 4 days a week,Commute to classes,sdv-pii-bwr4t,sdv-pii-eslhy,D1,KR MRT,LT27,NaN,10:50:51 AM,10 - 15 minutes,Neutral,Satisfied,Satisfied,Neutral,Satisfied,4,D2,sdv-pii-pve05,COM3,NaN,01:57:44 PM,15 - 20 minutes,Neutral,Satisfied,Very Satisfied,Neutral,Satisfied,4.0,D2,KR MRT,Opp KR MRT,Wednesday,NaN,10 - 15 minutes,Dissatisfied,Satisfied,NaN,Satisfied,NaN,4.0,2nd,5th,1st,5th,4th,3rd,4th,6th,4th,5th,1st,sdv-pii-qvtae,Frequently,1st,4th,6th,5th,2nd,4th,Yes,4,"No, service is consistent",sdv-pii-xojqa,NaN
2,10/01/2024 22:25:53,Undergraduate student,3 - 4 days a week,Commute to classes,sdv-pii-4nbco,sdv-pii-rdu87,D2,KR MRT,UTown,NaN,07:43:41 AM,15 - 20 minutes,Neutral,Neutral,Very Satisfied,Neutral,Satisfied,5,A2,sdv-pii-sz7il,PGP Foyer,NaN,11:37:05 AM,10 - 15 minutes,Dissatisfied,Satisfied,Very Satisfied,Neutral,Satisfied,NaN,D1,NaN,NaN,NaN,sdv-pii-ag5m7,NaN,Dissatisfied,Dissatisfied,Very Satisfied,Neutral,Satisfied,NaN,2nd,1st,1st,5th,5th,3rd,3rd,6th,5th,4th,1st,sdv-pii-85bfx,Frequently,2nd,4th,6th,5th,2nd,2nd,No,4,"Yes, service improves/worsens (please specify ...",sdv-pii-tonzs,sdv-pii-oclba
3,10/17/2024 03:43:29,Undergraduate student,1 - 2 days a week,Commute to classes,sdv-pii-rj915,sdv-pii-ket94,A2,UTown,CLB,"Tuesday, Thursday",10:23:07 AM,15 - 20 minutes,Dissatisfied,Neutral,Very Satisfied,Dissatisfied,Dissatisfied,5,NaN,sdv-pii-3ijdw,CLB,NaN,10:56:48 AM,20 - 30 minutes,Very dissatisfied,NaN,Satisfied,Very dissatisfied,Satisfied,4.0,NaN,KR MRT,S17,NaN,NaN,< 5 minutes,NaN,NaN,NaN,NaN,NaN,NaN,4th,3rd,3rd,5th,5th,1st,4th,6th,5th,1st,4th,sdv-pii-ur41g,Occasionally,1st,4th,6th,6th,3rd,3rd,No,3,"No, service is consistent",sdv-pii-cvr5x,NaN
4,10/06/2024 13:33:06,Undergraduate student,1 - 2 days a week,Commute to classes,sdv-pii-ieb2f,sdv-pii-k5fu5,A1,KR MRT,LT27,NaN,03:47:59 PM,10 - 15 minutes,Dissatisfied,Satisfied,Satisfied,Neutral,Satisfied,4,NaN,sdv-pii-9xn05,CLB,NaN,03:07:52 PM,15 - 20 minutes,Very dissatisfied,Satisfied,Satisfied,Satisfied,Satisfied,5.0,E,UTown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,